# 1. Ödev

Bu ödevde herhangi bir şekilde varolan sofistike kütüphaneler kullanılamaz. Sadece rastgele sayı üretmek için kütüphane kullanabilirsiniz. Dokümanları vektörize etmek, uzaklık hesapları yapmak vs. gibi her şey için gereken kodu kendiniz yazmalısınız.

Önemli not: ChatGpt'ye ben de kod yazdırabiliyorum :) ChatGPT'den alınan cevaplar kopya muamelesi görecektir.

Ödeviniz Python 3 ile implement ediniz

**Soru 1** Bu soruda locality sensitive hashing (LSH) metodu yazacaksınız. Metodun çağrımı şu şekildedir: lsh(klasor_adi, k, b, r, similarity_threshold). 
- klasor_adi indeksin kurulacağı dosyaları tutan klasörün adresidir. Klasörün içinde birçok txt dosyası var olduğunu varsayabilirsiniz.
- k 'k-shingle'in k değeridir. 
- b band sayısının değeridir.
- r her banddaki satır (row) sayısıdır. 

Verilen  klasördeki dosyaları okuyarak LSH sistemi kurun. Bunun için her kelime bir token olacaktır. Öncelikle her kelime küçük harflere çevrilmeli ve noktalama işaretleri çıkarılmalı. Eğer k = 2 ise, “Ali okula gitti.” cümlesindeki ‘shingle’lar “ali okula” ve “okula gitti” olmalı. 

MinHash imzaları için 10 tane hash fonksiyonu kullanınız. 

Not: Sorunun cevabını kontrol ederken ben size verilen dosyalar ile aynı formatta olan farklı dosyalar kullanacağım.


In [1]:
def zero_vec(length): # length uzunluğunda 0'lardan oluşan bir vektör döndürür
    vec = []
    for i in range(length):
        vec.append(0)
    return vec

In [2]:
def min_hash(index_range, num_of_hash_func):
    from sympy import randprime
    original_indexes = list(range(index_range))
    
    hash_functions = []
    for i in range(num_of_hash_func):
        a = randprime(1, 100000)
        b = randprime(1, 100000)
        if [a, b] not in hash_functions:
            hash_functions.append([a, b])
        else:
            i -= 1
        
    new_indexes = []

    for i in range(num_of_hash_func):
        a = hash_functions[i][0]
        b = hash_functions[i][1]

        hashed_list = [((a*x + b) % index_range**3) % index_range for x in original_indexes] 
        shuffled_list = [None] * index_range
        for i, index in enumerate(hashed_list):
            shuffled_list[index] = original_indexes[i]
        new_indexes.append(shuffled_list)
    
    return new_indexes

In [3]:
def create_hash_functions(num_hash_functions, num_of_shingles):
    import random
    hash_functions = []
    for i in range(num_hash_functions):
        a = random.randint(1, num_of_shingles-1)
        b = random.randint(1, 10000000)
        if [a, b] not in hash_functions:
            hash_functions.append([a, b])
        else:
            i -= 1
        
    return hash_functions

In [4]:
def transpose_dict(dict):
    matrix = []
    for i in dict.keys():
        matrix.append(dict[i])

    transposed = []
    for i in range(len(matrix[0])):
        temp = []
        for j in range(len(matrix)):
            temp.append(matrix[j][i])
        transposed.append(temp)

    transposed_dict = {}
    
    for key in range(1,len(transposed)+1):
        transposed_dict[key] = transposed[key-1]
    del transposed 
    del matrix
    return transposed_dict

-------------------YENİ--------------------

In [11]:
def lsh(klasor_adi, k, b, r):
    import os
    import string
    import sys
    import random

    matrix_represent = dict() # shingle'larin matrixteki temsilini tutacak

    dosya_isimleri = os.listdir(klasor_adi) # klasorun icindeki tum dosyalarin adini aliyoruz

    # tüm dosyaları tek tek oku
    for dosya in dosya_isimleri: 
        content = open(klasor_adi + "/" + dosya, "r").read()
        content = content.replace("\n", " ")
        content = content.lower()
        content = content.translate(str.maketrans('', '', string.punctuation))
        content = " ".join(content.split())        
        content = content.split(" ")
        for i in range(len(content) - k):
            shingle =  content[i:i + k]
            shingle = " ".join(shingle)
            if shingle not in matrix_represent.keys():
                vec = zero_vec(len(dosya_isimleri))
                vec[dosya_isimleri.index(dosya)] = 1
                matrix_represent[shingle] = vec
            else:
                matrix_represent[shingle][dosya_isimleri.index(dosya)] = 1

    num_of_shingles = len(matrix_represent.keys()) # shingle sayisi

    different_permutations = min_hash(num_of_shingles, 10) # farklı index permütasyonları oluşturuluyor

    signature_matrix = dict() # signature matrix oluşturuluyor
    transposed_matrix = transpose_dict(matrix_represent) # matrix_represent transpozu alınıyor

    for permutation in different_permutations: # her bir permütasyon için 
        i = different_permutations.index(permutation)
        temp = []
        for j in transposed_matrix.keys(): # her bir doküman için
            for index in permutation: # her bir permütasyonun içindeki indexler için
                if transposed_matrix[j][index] == 1:
                    temp.append(index)
                    break 
            signature_matrix[i] = temp

    del different_permutations
    # kod buraya kadar çalışıyor

    # signature matrix'i b ve r değerlerine göre bölüyoruz
    signature_matrix_bolunmus = dict()
    for i in range(b): # signature matrix'i b'ye bölüyoruz
        signature_matrix_bolunmus[i] = []
    
        for doc in range(len(signature_matrix[0])): # her bir doküman için
            temp = []
            for j in range(r):
                temp.append(signature_matrix[i+j][doc])
            signature_matrix_bolunmus[i].append(temp)
    
    
    buckets = dict() # bucket'lar oluşturuluyor

    for i in range(b): # her bir b için
        buckets[i] = dict()
        for j in range(len(signature_matrix_bolunmus[i])): # her bir doküman için
            numbers = signature_matrix_bolunmus[i][j]
            concatenated_str = ''.join(str(num) for num in numbers)
            unicode_values = [ord(char) for char in concatenated_str]
            unicode_values_concatenated = ''.join(str(num) for num in unicode_values)
            if unicode_values_concatenated not in buckets[i].keys():
                buckets[i][unicode_values_concatenated] = [dosya_isimleri[j]]
            else:
                buckets[i][unicode_values_concatenated].append(dosya_isimleri[j])

    return buckets, matrix_represent, dosya_isimleri




    

    

----------------------ESKİ-----------------------

In [337]:
def lsh(klasor_adi, k, b, r):
    import os
    import string
    import sys
    import random

    matrix_represent = dict() # shingle'larin matrixteki temsilini tutacak

    dosya_isimleri = os.listdir(klasor_adi) # klasorun icindeki tum dosyalarin adini aliyoruz

    # tüm dosyaları tek tek oku
    for dosya in dosya_isimleri: 
        content = open(klasor_adi + "/" + dosya, "r").read()
        content = content.replace("\n", " ")
        content = content.lower()
        content = content.translate(str.maketrans('', '', string.punctuation))
        content = " ".join(content.split())        
        content = content.split(" ")
        for i in range(len(content) - k):
            shingle =  content[i:i + k]
            shingle = " ".join(shingle)
            if shingle not in matrix_represent.keys():
                vec = zero_vec(len(dosya_isimleri))
                vec[dosya_isimleri.index(dosya)] = 1
                matrix_represent[shingle] = vec
            else:
                matrix_represent[shingle][dosya_isimleri.index(dosya)] = 1

    # shingle'larin sırasını karıştırıyoruz
    keys = list(matrix_represent.keys())
    random.shuffle(keys)

    temp_dict = {}
    for key in keys:
        temp_dict[key] = matrix_represent[key]
    del matrix_represent
    matrix_represent = temp_dict
    # ------------------------------

    num_of_shingles = len(matrix_represent.keys()) # shingle sayisi

    different_permutations = list() # farklı permütasyonları tutacak
    for i in range(10):
        temp = list(range(num_of_shingles))
        random.shuffle(temp)
        different_permutations.append(temp)
   

    signature_matrix = dict() # signature matrix oluşturuluyor
    transposed_matrix = transpose_dict(matrix_represent) # matrix_represent transpozu alınıyor

    for permutation in different_permutations: # her bir permütasyon için 
        i = different_permutations.index(permutation)
        temp = []
        for j in transposed_matrix.keys(): # her bir doküman için
            for index in permutation: # her bir permütasyonun içindeki indexler için
                if transposed_matrix[j][index] == 1:
                    temp.append(index)
                    break 
            signature_matrix[i] = temp

    del different_permutations, temp_dict
    # kod buraya kadar çalışıyor

    # signature matrix'i b ve r değerlerine göre bölüyoruz
    signature_matrix_bolunmus = dict()
    for i in range(b): # signature matrix'i b'ye bölüyoruz
        signature_matrix_bolunmus[i] = []
    
        for doc in range(len(signature_matrix[0])): # her bir doküman için
            temp = []
            for j in range(r):
                temp.append(signature_matrix[i+j][doc])
            signature_matrix_bolunmus[i].append(temp)
    
    
    buckets = dict() # bucket'lar oluşturuluyor

    for i in range(b): # her bir b için
        buckets[i] = dict()
        for j in range(len(signature_matrix_bolunmus[i])): # her bir doküman için
            numbers = signature_matrix_bolunmus[i][j]
            concatenated_str = ''.join(str(num) for num in numbers)
            unicode_values = [ord(char) for char in concatenated_str]
            unicode_values_concatenated = ''.join(str(num) for num in unicode_values)
            if unicode_values_concatenated not in buckets[i].keys():
                buckets[i][unicode_values_concatenated] = [dosya_isimleri[j]]
            else:
                buckets[i][unicode_values_concatenated].append(dosya_isimleri[j])

    return buckets, signature_matrix_bolunmus






    

In [59]:
a,b = lsh("docs2", 2,5,2)

Permutation:  [4135, 3498, 2861, 2224, 1587, 950, 313, 3990, 3353, 2716, 2079, 1442, 805, 168, 3845, 3208, 2571, 1934, 1297, 660, 23, 3700, 3063, 2426, 1789, 1152, 515, 4192, 3555, 2918, 2281, 1644, 1007, 370, 4047, 3410, 2773, 2136, 1499, 862, 225, 3902, 3265, 2628, 1991, 1354, 717, 80, 3757, 3120, 2483, 1846, 1209, 572, 4249, 3612, 2975, 2338, 1701, 1064, 427, 4104, 3467, 2830, 2193, 1556, 919, 282, 3959, 3322, 2685, 2048, 1411, 774, 137, 3814, 3177, 2540, 1903, 1266, 629, 4306, 3669, 3032, 2395, 1758, 1121, 484, 4161, 3524, 2887, 2250, 1613, 976, 339, 4016, 3379, 2742, 2105, 1468, 831, 194, 3871, 3234, 2597, 1960, 1323, 686, 49, 3726, 3089, 2452, 1815, 1178, 541, 4218, 3581, 2944, 2307, 1670, 1033, 396, 4073, 3436, 2799, 2162, 1525, 888, 251, 3928, 3291, 2654, 2017, 1380, 743, 106, 3783, 3146, 2509, 1872, 1235, 598, 4275, 3638, 3001, 2364, 1727, 1090, 453, 4130, 3493, 2856, 2219, 1582, 945, 308, 3985, 3348, 2711, 2074, 1437, 800, 163, 3840, 3203, 2566, 1929, 1292, 655, 18, 3695, 305

In [62]:
for key in a.keys():
    for key2 in a[key].keys():
        if len(a[key][key2]) > 1:
            print(a[key][key2])

['1000.txt', '1029.txt']
['1001.txt', '1011.txt']
['1021.txt', '1025.txt']
['1001.txt', '1002.txt', '1014.txt', '1028.txt']
['1009.txt', '1010.txt']
['1001.txt', '1002.txt']
['1005.txt', '1026.txt']
['1009.txt', '1013.txt']
['1000.txt', '1007.txt']
['1001.txt', '1002.txt', '1009.txt', '1013.txt', '1015.txt', '1020.txt']
['1004.txt', '1005.txt', '1008.txt', '1012.txt']
['1006.txt', '1014.txt', '1019.txt']
['1000.txt', '1005.txt', '1016.txt']
['1001.txt', '1007.txt', '1015.txt']
['1003.txt', '1006.txt', '1010.txt', '1011.txt', '1014.txt', '1019.txt', '1020.txt']
['1008.txt', '1009.txt']


Yukarıda aynı bucketta olanlar çalışıyor.

**Soru 2)**  Önceki soruda hazırladığınız LSH indeksini kullanarak verilen bir dosyanın muhtemel benzerleri olan dosyaları ve o dokümanlar ile olan jaccard benzerliği sıralı bir şekilde ekrana yazdıran bir method yazınız.  En çok benzeyen dokümanlar ilk yazılmalı. Çıktının formatı aşağıdaki örnekteki gibi olmalı.  Örnekte sadece 2 dosya döndüğü varsayılmıştır.

a.txt 0.7
b.txt 0.65
Toplam 2 dosya bulunmuştur.


In [124]:
def jaccard_similarity(doc1, doc2):
    intersection = 0
    for i in range(len(doc1)):
        if doc1[i] == doc2[i]:
            intersection += 1
    
    union = len(doc1) + len(doc2) - intersection
    return intersection / union

In [158]:
# Cevabınız buraya yazınız. Eğer ihtiyaç duyarsanız metoda başka parametreler ekleyebilirsiniz
def query(input_file, lsh_output, matrix_representation, doc_names):
    candidate_pairs = [input_file]
 
    for band in lsh_output.keys(): 
        for bucket in lsh_output[band].keys():
            if input_file in lsh_output[band][bucket]:
                if len(lsh_output[band][bucket]) > 1:
                    for doc in lsh_output[band][bucket]:
                        if doc != input_file and doc not in candidate_pairs:
                            candidate_pairs.append(doc)

    doc_indexes = []
    for doc in candidate_pairs:
        doc_indexes.append(doc_names.index(doc))
    
    # matrix_representation'da her key için bir vektör var. Bu vektörlerde sadece doc_indexes olanlar kalsın. Diğerleri silinsin.
    temp_dict = {}
    for key in matrix_representation.keys():
        temp = []
        for index in doc_indexes:
            temp.append(matrix_representation[key][index])
        temp_dict[key] = temp
    matrix_representation = temp_dict

    doc_values = {}

    for doc in candidate_pairs:
        doc_values[doc] = []

    for shingle in matrix_representation.keys():
        for doc in candidate_pairs:
            doc_values[doc].append(matrix_representation[shingle][candidate_pairs.index(doc)])

    remove_indexes = []
    for idx in range(len(doc_values[input_file])):
        is_all_zero = True
        for doc in candidate_pairs:
            if doc_values[doc][idx] != 0:
                is_all_zero = False
                break
        
        if is_all_zero == True:
            remove_indexes.append(idx)

    for doc in candidate_pairs:
        temp = []
        for idx in range(len(doc_values[doc])):
            if idx not in remove_indexes:
                temp.append(doc_values[doc][idx])
        doc_values[doc] = temp

    similarities = dict()
    for doc in candidate_pairs[1:]:
        similarity = round(jaccard_similarity(doc_values[input_file], doc_values[doc]),3)
        similarities[doc] = similarity

    similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    for sim in similarities:
        print(sim[0], sim[1], end=" ") 

    print("Toplam " + str(len(similarities)) + " adet benzer doküman bulundu.")

In [161]:
lsh_output, shingle_matrix, doc_names = lsh("docs2", 2,5,2)

In [162]:
a = query("1005.txt", lsh_output, shingle_matrix, doc_names)


1000.txt 0.905 1006.txt 0.88 1008.txt 0.876 1003.txt 0.871 1001.txt 0.859 1007.txt 0.859 1012.txt 0.858 1016.txt 0.846 1011.txt 0.838 1002.txt 0.836 1010.txt 0.831 1013.txt 0.831 1014.txt 0.831 1019.txt 0.829 1018.txt 0.823 1059.txt 0.819 1085.txt 0.813 1009.txt 0.81 1044.txt 0.805 1015.txt 0.802 1076.txt 0.799 1094.txt 0.794 1052.txt 0.792 1060.txt 0.792 1099.txt 0.792 1096.txt 0.788 1084.txt 0.785 1047.txt 0.78 1043.txt 0.77 Toplam 29 adet benzer doküman bulundu.


**Doğru çalışıyor buraya kadar**

**Soru 3)** Bu soruda ödev ile verilen  metin dosyalarını kullanarak LSH indeksi kurup farklı k, b, ve r değerlerinin sonuç üzerindeki etkisine bakacaksınız.
Spesifik olarak yapacağınız şunlar.
- Ödevde verilen docs klasöründeki dokümanları b=5, r=4 iken k=2,3,4 değerlerini kullanarak 3 farklı LSH indeksi kurunuz. Ardından ödevde verilen sorgu.txt dosyasını bu 3 ayrı indekste aratıp toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz. 
- k=3, r=4 iken b değerini 3,4,5 olarak değiştiriniz ve birinci şıktaki gibi toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz. 
- k=3, r=4 iken r değerini 3,4,5 olarak değiştiriniz ve birinci şıktaki gibi toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz.

Not:  Bu soruda ikinci sorudaki metotta ufak bir değişikliğe gidip ekrana bir şey bastırmadan sadece  kaç doküman bulunduğunu dönen bir metot olarak tanımlayın. Bu yeni metodu aşağıda 3. soru için tanımlanan yerde yapın.

In [16]:
# 3. sorunun cevabı için ayrılan yer

**Soru 4**	Bu soruda MeanShift algoritmasını implement edeceksiniz. Yazacağınız metot verilen klasördeki her dosyayı meanShift algoritmasına ve radius uzaklık parametresine göre kümeler ve çıktı olarak kümeleri print eder. Algoritma en başta klasör içindeki her dosya için bir ağırlık merkezi başlatır. Çıktısı şu formatta olmalıdır.

Cluster-0: a,b,c

Cluster-1: d,e,f 

...........

Cluster-n: h
 
Yukarıdaki örnekte a,b,c, vs. verilen klasördeki dosyaların isimleridir. Dokümanların vektörel gösterimi şöyle olmalıdır: Tüm dokümanlarda geçen en popüler 1000 kelimeyi bulun. Sonra her kelimenin varlığı/yokluğunu birer öznitelik olarak gösterin. Örneğin bir doküman “a b c d f” ise ve b, d ve s en popüler ‘kelimeler’ ise, bu dokümanın gösterimi [1,1,0] olmalıdır. Uzaklık için kosinüs uzaklığını kullanın.



In [17]:
def meanShift (String folder_name, float radius):

SyntaxError: invalid syntax (1091480408.py, line 1)

**Soru 5** 4. soruda tanımladığınız metodu kullanarak ödev ile beraber verilen docs klasöründeki dosyaları cluster ediniz. Bunun için radius değerinini 0.1’den başlayıp 0.05 arttırarak 0.95’e kadar tüm değerlerde deneyin ve elde ettiğiniz cluster sayısını gösteriniz. Bunun için x ekseninde Radius değeri, y ekseninde cluster sayısı olan bir figür oluşturunuz.